In [1]:
from utils import *

# Code Assistant
This agent is responsible for creating and executing python files and connecting to live data.

In [2]:
class ExecutePyFile(OpenAISchema):
    """Run existing python file from local disc."""
    file_name: str = Field(
        ..., description="The path to the .py file to be executed."
    )

    def run(self):
        """Executes a Python script at the given file path and captures its output and errors."""
        try:
            result = subprocess.run(
              ['python3', self.file_name],
              text=True,
              capture_output=True,
              check=True
            )
            return result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred: {e.stderr}"

class File(OpenAISchema):
    """
    Python file with an appropriate name, containing code that can be saved and executed locally at a later time. This environment has access to all standard Python packages and the internet.
    """
    chain_of_thought: str = Field(...,
        description="Think step by step to determine the correct actions that are needed to be taken in order to complete the task.")
    file_name: str = Field(
        ..., description="The name of the file including the extension"
    )
    body: str = Field(..., description="Correct contents of a file")

    def run(self):
        with open(self.file_name, "w") as f:
            f.write(self.body)

        return "File written to " + self.file_name
    
import requests
from pydantic import Field
from typing import Union

class APITool(OpenAISchema):
    """
    Tool to access live data from various free APIs.
    """
    api_url: str = Field(
        ..., description="The URL of the API you want to access."
    )
    api_key: str = Field(
        None, description="API key if required for authentication."
    )
    method: str = Field(
        'GET', description="HTTP method to use for the API request (GET, POST, etc.)."
    )
    params: dict = Field(
        {}, description="Parameters to include in the API request."
    )

    def run(self):
        """
        Makes an API request and returns the response.
        """
        headers = {'Authorization': f'Bearer {self.api_key}'} if self.api_key else {}

        try:
            response = requests.request(
                method=self.method,
                url=self.api_url,
                params=self.params,
                headers=headers
            )
            response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code

            return response.json()  # Assuming the response is in JSON format
        except requests.exceptions.RequestException as e:
            return f"An error occurred: {str(e)}"

In [3]:
code_assistant_funcs = [File, ExecutePyFile, APITool]

code_assistant = client.beta.assistants.create(
    name='Code Assistant Agent',
    instructions="As a top-tier programming AI, you are adept at creating accurate Python scripts. You will properly name files and craft precise Python code with the appropriate imports to fulfill the user's request. Ensure to execute the necessary code before responding to the user.",
    model="gpt-4-1106-preview",
    tools=[{"type": "function", "function": File.openai_schema},
         {"type": "function", "function": ExecutePyFile.openai_schema},]
)

# User Proxy
This agent facilitates the conversation between the user and other agents, ensuring successful completion of the task.

In [4]:
import subprocess
from enum import Enum
from pydantic import PrivateAttr
from typing import Literal

agents_and_threads = {
    "code_assistant": {
        "agent": code_assistant,
        "thread": None,
        "funcs": code_assistant_funcs
    }
}

class SendMessage(OpenAISchema):
    """Send messages to other specialized agents in this group chat."""
    recepient:Literal['code_assistant'] = Field(..., description="code_assistant is a world class programming AI capable of executing python code.")
    message: str = Field(...,
        description="Specify the task required for the recipient agent to complete. Focus instead on clarifying what the task entails, rather than providing detailed instructions.")

    def run(self):
        recepient = agents_and_threads[self.recepient]
        # if there is no thread between user proxy and this agent, create one
        if not recepient["thread"]:
            recepient["thread"] = client.beta.threads.create()
        message = get_completion(message=self.message, **recepient)

        return message

In [5]:
user_proxy_tools = [SendMessage]

user_proxy = client.beta.assistants.create(
  name='User Proxy Agent',
  instructions="""As a user proxy agent, your responsibility is to streamline the dialogue between the user and specialized agents within this group chat.
Your duty is to articulate user requests accurately to the relevant agents and maintain ongoing communication with them to guarantee the user's task is carried out to completion.
Please do not respond to the user until the task is complete, an error has been reported by the relevant agent, or you are certain of your response.""",
  model="gpt-4-1106-preview",
  tools=[
      {"type": "function", "function": SendMessage.openai_schema},
  ],
)

# ToT assistant

In [6]:
assistant_tot = client.beta.assistants.create(
    name="Multi-expert assistant",
    instructions="Engage in a collaborative thinking exercise using the Tree of Thoughts technique. Begin by presenting a question and theme, and then envision three distinct experts, each specializing in a different field. They must critique and try to find drawbacks in the answer and improve it, but not deviate from the question and answer format. Each expert will provide concise and informative critiques, starting with the initial step of their thought process. After sharing their first step, experts will collectively move to the next step and repeat the process iteratively. If an expert realizes an error at any point, they exit the discussion. Ultimately, the group must converge on a single, well-reasoned answer, which is separated from all discussion with a line of *.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

def create_tot(message, assistant):
    thread = client.beta.threads.create()
    
    message = client.beta.threads.messages.create(
                      thread_id=thread.id,
                      role="user",
                      content=message,
                    )
        
    run = client.beta.threads.runs.create(
              thread_id=thread.id,
              assistant_id=assistant.id,
            )
    
    run = wait_on_run(run, thread)
    
    messages = client.beta.threads.messages.list(
                  thread_id=thread.id
                )
    
    for message in reversed(messages.data):
        print(message.role + ": " + message.content[0].text.value)

# Autogen Example Questions:

Example: <br>1. How did the bitcoin price change in 7 days and why?<br>
         2. Show and save the plot of the price of bitcoin in last 7 days, with window name `Bitcoin Price Distribution`.

In [ ]:
thread = client.beta.threads.create()
while True:
    user_message = input("User: ")

    message = get_completion(user_message, user_proxy, user_proxy_tools, thread)
    
    wprint(f"\033[34m{user_proxy.name}: ", message,'\033[0m')    
    
    create_tot('Question:' + user_message + '\nAnswer: ' + message, assistant_tot)

User: How did the bitcoin price change in 7 days and why?
Function(arguments='{"recepient":"crypto_assistant","message":"Wh
at is the 7-day price change for Bitcoin?"}', name='SendMessage') 
SendMessage: Error: 1 validation error for SendMessage recepient   Input should be
'code_assistant' [type=literal_error, input_value='crypto_assistant',
input_type=str]     For further information visit
https://errors.pydantic.dev/2.5/v/literal_error 
Function(arguments='{"recepient":"code_assistant","message":"What
is the 7-day price change for Bitcoin?"}', name='SendMessage') 
Function(arguments='{\n  "chain_of_thought": "The task requires
retrieving the 7-day price change for Bitcoin which typically involves
fetching data from a cryptocurrency API. We\'ll use a Python script to
make an API request to a cryptocurrency data provider to get the
current and past price of Bitcoin, then calculate the 7-day price
change.",\n  "file_name": "bitcoin_7day_change.py",\n  "body": "import
requests\\nimport d

User: Show and save the plot of the price of bitcoin in last 7 days, with window name `Bitcoin Price Distribution`.
Function(arguments='{"recepient":"code_assistant","message":"Can
you generate a plot of the Bitcoin price over the last 7 days and save
it with the window name \'Bitcoin Price Distribution\'?"}',
name='SendMessage') 
Function(arguments='{"chain_of_thought":"To generate a plot of
the Bitcoin price over the last 7 days, we\'ll use matplotlib to
create a time-series plot. We\'ll fetch the Bitcoin price data using
the same CoinGecko API as before. Then we\'ll parse the data to
extract the dates and prices, and use matplotlib to plot and save the
figure with the specified window name, \'Bitcoin Price
Distribution\'.","file_name":"plot_bitcoin_price.py","body":"import
requests\\nimport matplotlib.pyplot as plt\\nimport matplotlib.dates
as mdates\\nfrom datetime import datetime\\n\\n# Fetch the Bitcoin
price data\\nurl = \'https://api.coingecko.com/api/v3/coins/bitcoin/ma
rket_c